In [21]:
#!pip install opencv-python

## images folder 안의 이미지 불러오기

In [2]:
import os
import pandas as pd
import cv2
from skimage.metrics import structural_similarity as ssim

In [4]:
photo_list = []

for f in os.listdir('../imageGenerate/build/images'):
    if 'png' in f:
        photo_list.append(f)

In [5]:
photo_size = list(map(lambda x: os.path.getsize('../imageGenerate/build/images' + '/' + x), photo_list))

In [6]:
# Find Same Photos
fsp = pd.DataFrame({'filename_raw':photo_list, 'size':photo_size})

print('사진의 갯수 :', len(fsp))

사진의 갯수 : 11


In [7]:
fsp

,filename_raw,size
0,8.png,199698
1,9.png,212742
2,11.png,205413
3,10.png,179631
4,4.png,223850
5,5.png,205257
6,7.png,205413
7,6.png,183736
8,2.png,213018
9,3.png,200913


### 1. 겹치는 파일명 확인 
(image generate 하면 다 다를 것이기 때문에 필요 없을 가능성이 많음)

In [8]:
# 겹치는 파일명 있을 시 뒤에 숫자 제거
import re   # 정규표현식
com = re.compile(' \d')
fsp['filename'] = list(map(lambda x: com.sub('', x), photo_list))

In [9]:
pvc = pd.DataFrame({'filename':fsp['filename'].value_counts().index, 'fn_counts':fsp['filename'].value_counts().values})   
psvc = pd.DataFrame({'size':fsp['size'].value_counts().index, 'size_counts':fsp['size'].value_counts().values})   

fsp = pd.merge(fsp, pvc, how = 'left', on = 'filename')
fsp = pd.merge(fsp, psvc, how = 'left', on = 'size')

fsp.sample(2)

,filename_raw,size,filename,fn_counts,size_counts
10,1.png,207157,1.png,1,1
8,2.png,213018,2.png,1,1


In [10]:
for i in range(len(fsp)):
    if (fsp['fn_counts'][i] > 1) & (fsp['size_counts'][i] == 1):
        print(i)

In [11]:
# Find Same Phto_Not Same Name
fsp_nsn = fsp.sort_values(['filename_raw'], ascending = False).drop_duplicates(['filename'], keep = 'first')

In [12]:
print('남은 사진의 갯수 : {}\n지워진 사진의 갯수 : {}'.format(len(fsp_nsn), len(fsp)-len(fsp_nsn)))

남은 사진의 갯수 : 11
지워진 사진의 갯수 : 0


In [13]:
pvc_nsn = pd.DataFrame({'filename':fsp_nsn['filename'].value_counts().index, 'fn_counts_nsn':fsp_nsn['filename'].value_counts().values})   
psvc_nsn = pd.DataFrame({'size':fsp_nsn['size'].value_counts().index, 'size_counts_nsn':fsp_nsn['size'].value_counts().values})   

fsp_nsn = pd.merge(fsp_nsn, pvc_nsn, how = 'left', on = 'filename')
fsp_nsn = pd.merge(fsp_nsn, psvc_nsn, how = 'left', on = 'size')

In [14]:
fsp_nsn[fsp_nsn['fn_counts_nsn']!=1]

,filename_raw,size,filename,fn_counts,size_counts,fn_counts_nsn,size_counts_nsn


In [15]:
print('사이즈 겹치는 사진의 갯수 :', len(fsp_nsn[fsp_nsn['size_counts_nsn']!=1]))
print('중복 사이즈의 갯수 :', len(psvc_nsn[psvc_nsn['size_counts_nsn']>1]))

사이즈 겹치는 사진의 갯수 : 2
중복 사이즈의 갯수 : 1


### 사이즈가 같은 이미지 비교
1) 사이즈가 같은 두 이미지를 불러온다.

2) 이미지를 array로 변환했을 때 구조가 같다면 두 이미지에 차이가 있는지 확인한다. 

3) 만약 두개의 이미지 구조가 같은데 차이가 존재한다면 그 사진은 직접 확인하기로 한다.

In [19]:
# 삭제될 사진의 리스트
delete = []


for i in range(len(psvc_nsn)):
    
    # 중복된 크기(size)가 있는 경우
    if psvc_nsn['size_counts_nsn'][i] == 2:
        
        # 그 크기에 해당하는 사진을 불러온다. 
        temp = fsp_nsn[fsp_nsn['size']==psvc_nsn['size'][i]].reset_index(drop = True).sort_values(['filename'])
        
        # 사진 읽기
        imageA = cv2.imread('../imageGenerate/build/images/'+temp['filename_raw'][0])
        imageB = cv2.imread('../imageGenerate/build/images/'+temp['filename_raw'][1])
    
        # 이미지를 grayscale로 변환
        grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
        grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
        
        # 이미지의 구조가 같다면 이미지 비교
        if len(grayA)==len(grayB):
            (score, diff) = ssim(grayA, grayB, full=True)
            
            # 차이가 없다면 하나는 delete에 넣어주기
            if score == 1:
                delete.append(temp['filename_raw'][1])
            
            # 구조가 같지만 차이가 존재한다면 직접 확인하기     
            else:
                print('확인해보시오! : ', temp['filename_raw'][0], '/', temp['filename_raw'][1], f'(score : {score})')

In [20]:
# 삭제해야 할 이미지 출력!
print(delete)

['11.png']
